In [ ]:
from google.colab import drive
#drive.flush_and_unmount()
 
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os, sys
sys.path.insert(0,"/content/drive/My Drive/Colab_PackagesandLibraries")

import pandas as pd
import librosa
import glob 
import librosa.display
import random

import tensorflow as tf
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

from keras.utils.np_utils import to_categorical


import time

import numpy as np
from sklearn import metrics 

from sklearn.datasets import make_regression
from sklearn.preprocessing import StandardScaler
from sklearn import metrics
from sklearn.model_selection import train_test_split, GridSearchCV

from sklearn.preprocessing import LabelEncoder

#import wget
#!pip install --target="/content/drive/My Drive/Colab_PackagesandLibraries" pytictoc
# Install files to spesific directory
#url = "https://ndownloader.figshare.com/files/10853306"
#output_directory = "/content/drive/My Drive/Datasets/warblrb10k/warblrb10k_public_metadata_2018.csv"
#wget.download(url, out=output_directory)

#url = "https://ndownloader.figshare.com/files/10853303"
#output_directory = "/content/drive/My Drive/Datasets/ff1010bird/ff1010bird_metadata_2018.csv"
#wget.download(url, out=output_directory)

# load data
#!wget https://archive.org/download/ff1010bird/ff1010bird_wav.zip

#!wget https://archive.org/download/warblrb10k_public/warblrb10k_public_wav.zip

# Unzip folder to spesific directory
#!unzip ff1010bird_wav.zip -d "/content/drive/My Drive/Datasets/ff1010bird"

#!unzip warblrb10k_public_wav.zip -d "/content/drive/My Drive/Datasets/warblrb10k"

In [ ]:
datafolder = "/content/drive/My Drive/Datasets/ff1010bird/"
loc = datafolder+"wav/"
metadata = pd.read_csv("/content/drive/My Drive/Datasets/ff1010bird/ff1010bird_metadata_2018.csv")
m,n = metadata.shape
P = 0.05
idx = np.random.permutation(m)
metadataTrain = metadata.iloc[idx[:round(P*m)]] 
metadataValidation = metadata.iloc[idx[round(P*m):round(m/10)]]
trainFiles = [loc+str(s)+".wav" for s in metadataTrain["itemid"]]
#print(trainFiles)
trainLabels = [str(s) for s in metadataTrain["hasbird"]]
#print(trainLabels)
validationFiles = [loc+str(s)+".wav" for s in metadataValidation["itemid"]]
#print(validationFiles)
validationLabels = [str(s) for s in metadataValidation["hasbird"]]
#print(validationLabels)
trainFiles = np.vstack((trainFiles, trainLabels)).T
#print(trainFiles)
validationFiles = np.vstack((validationFiles,validationLabels)).T
#print(validationFiles)



In [ ]:
#from pytictoc import TicToc
def extract_features(dir,
                     bands=40,frames=41):
  def _windows(data, window_size):
    start = 0
    while start < len(data):
      yield int(start), int(start + window_size)
      start += (window_size // 2)

          
  window_size = 512 * (frames - 1)
  features, labels = [], []
  for fn in dir:
    segment_log_specgrams, segment_labels = [], []
    sound_clip,sr = librosa.load(fn[0])
    label = fn[1]
    for (start,end) in _windows(sound_clip,window_size):
      if(len(sound_clip[start:end]) == window_size):
        signal = sound_clip[start:end]
        melspec = librosa.feature.melspectrogram(signal,n_mels=bands)
        logspec = librosa.amplitude_to_db(melspec)
        logspec = logspec.T.flatten()[:, np.newaxis].T
        segment_log_specgrams.append(logspec)
        segment_labels.append(label)
            
    segment_log_specgrams = np.asarray(segment_log_specgrams).reshape(
                len(segment_log_specgrams),bands,frames,1)
    segment_features = np.concatenate((segment_log_specgrams, np.zeros(
                np.shape(segment_log_specgrams))), axis=3)
    for i in range(len(segment_features)): 
      segment_features[i, :, :, 1] = librosa.feature.delta(
                    segment_features[i, :, :, 0]) 
    if len(segment_features) > 0: # check for empty segments 
      features.append(segment_features)
      labels.append(segment_labels)
  return features, labels

#os.chdir("/content/drive/My Drive/Datasets/ff1010bird/wav/125557.wav")

#print(trainFiles[0][0])
sc, sr = librosa.load("/content/drive/My Drive/Datasets/ff1010bird/wav/191742.wav")
#ti = TicToc() # create TicToc instance
#ti.tic() # Start timer
#print(trainFiles[0][1])
trainfeatureVectors, trainVectorLabels=extract_features(trainFiles)
ValidationfeatureVectors, ValidationVectorLabels=extract_features(validationFiles)
#ti.toc() # Print elapsed time




In [63]:
trainVectorLabels = np.array(trainVectorLabels).flatten()
trainVectorLabels = trainVectorLabels.reshape(len(trainVectorLabels),1)
ValidationVectorLabels = np.array(ValidationVectorLabels).flatten()
ValidationVectorLabels = ValidationVectorLabels.reshape(len(ValidationVectorLabels),1)

trainfeatureVectors_flat = np.array(trainfeatureVectors).flatten()
trainfeatureVectors_flat = trainfeatureVectors_flat.reshape(int(trainfeatureVectors_flat.shape[0]/len(trainVectorLabels)),len(trainVectorLabels))

norm = np.linalg.norm(trainfeatureVectors_flat)
trainfeatureVectors_flat = trainfeatureVectors_flat/norm #########

print(trainfeatureVectors_flat)
print(trainfeatureVectors_flat.shape)
#trainVectorLabels_flat = trainfeatureVectors.flatten()
ValidationfeatureVectors_flat = np.array(ValidationfeatureVectors).flatten()
ValidationfeatureVectors_flat = ValidationfeatureVectors_flat.reshape(int(ValidationfeatureVectors_flat.shape[0]/len(ValidationVectorLabels)),len(ValidationVectorLabels))
norm = np.linalg.norm(ValidationfeatureVectors_flat)
ValidationfeatureVectors_flat = ValidationfeatureVectors_flat/norm #########
print(ValidationfeatureVectors_flat)
print(ValidationfeatureVectors_flat.shape)

#ValidationVectorLabels = trainfeatureVectors.flatten()

[[ 1.21848063e-05 -9.20912121e-06  1.74555051e-04 ...  5.74748122e-05
  -9.89949490e-05  6.22699317e-05]
 [ 6.06143039e-05  5.51667834e-05  1.62028690e-04 ...  5.91952539e-05
  -1.07809017e-04  6.58803147e-05]
 [ 6.52662509e-06  5.84280133e-05  1.13577403e-04 ...  3.00307855e-05
  -1.45454023e-04  3.00307855e-05]
 ...
 [ 1.76113497e-04  1.90961197e-05  1.56546946e-04 ...  8.73497155e-05
  -3.28699955e-04  8.98937804e-05]
 [ 1.98845229e-04  8.09629628e-05  2.62681626e-04 ...  7.78351024e-05
  -3.01176930e-04  8.11280226e-05]
 [ 2.73941145e-04  7.90689138e-05  2.43023817e-04 ... -2.24123620e-20
  -3.75275269e-04 -2.24123620e-20]]
(3280, 7680)
[[-3.41465913e-05  1.63758154e-05  3.46432544e-05 ... -1.61693860e-05
   5.99906694e-06 -2.24223989e-05]
 [-6.30704039e-05 -2.24223989e-05 -1.40611006e-04 ... -9.33300518e-06
  -1.53701374e-04 -7.33440962e-06]
 [-1.96741342e-04 -1.73227789e-05 -3.69395919e-04 ... -1.99872855e-05
  -3.15611515e-04 -5.08602979e-06]
 ...
 [-1.90184728e-04 -2.05503130e-

In [ ]:
# trainfeatureVectors = np.concatenate(trainfeatureVectors)
# repeat_num = len(trainfeatureVectors[1])
# trainfeatureVectors = np.concatenate(trainfeatureVectors)
# trainVectorLabels = np.concatenate(trainVectorLabels)
# trainVectorLabels = np.tile(trainVectorLabels, (repeat_num, 1))
# trainVectorLabels = np.concatenate(trainVectorLabels)
# ValidationfeatureVectors = np.concatenate(ValidationfeatureVectors)
# ValidationfeatureVectors = np.concatenate(ValidationfeatureVectors)
# ValidationVectorLabels = np.concatenate(ValidationVectorLabels)
# ValidationVectorLabels = np.tile(ValidationVectorLabels, (repeat_num, 1))

In [ ]:
#print(trainfeatureVectors.shape)
#print(trainVectorLabels.shape)
#print(ValidationfeatureVectors.shape)
#print(ValidationVectorLabels.shape)



In [65]:

model = tf.keras.Sequential(
    [
        tf.keras.Input(shape=(trainfeatureVectors_flat.shape[0])),
        # tf.keras.layers.experimental.RandomFourierFeatures(
        #     output_dim=4096, scale=10.0, kernel_initializer="gaussian"
        # ),
        tf.keras.layers.Dense(512, activation= "relu"),
        tf.keras.layers.Dense(256, activation = "linear"),
        tf.keras.layers.Dense(256, activation = "relu"),
        tf.keras.layers.Dense(256, activation = "relu"),
        tf.keras.layers.Dense(64, activation = "linear"),
        tf.keras.layers.Dense(64, activation = "relu"),
        tf.keras.layers.Dense(64, activation = "relu"),
        tf.keras.layers.Dense(32, activation = "relu"),
        tf.keras.layers.Dense(16, activation = "relu"),
        tf.keras.layers.Dense(1, activation = "sigmoid"),
    ]
)
model.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss=tf.keras.losses.BinaryCrossentropy(),
    metrics=[tf.keras.metrics.BinaryAccuracy()],
)



In [81]:
history = model.fit(trainfeatureVectors_flat.T.astype(np.float),trainVectorLabels.astype(np.float),batch_size = 512,epochs=50,verbose=0,shuffle=True)

model.summary()

# Evaluate random veriyor
test_loss, test_acc = model.evaluate(ValidationfeatureVectors_flat.T.astype(np.float), ValidationVectorLabels.astype(np.float))
predictions = model.predict(ValidationfeatureVectors_flat.T.astype(np.float))

# predictions = model.predict(ValidationfeatureVectors_flat.T.astype(np.float))

# print(test_acc)

print(predictions)
# print(ValidationVectorLabels)

Model: "sequential_32"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_266 (Dense)            (None, 512)               1679872   
_________________________________________________________________
dense_267 (Dense)            (None, 256)               131328    
_________________________________________________________________
dense_268 (Dense)            (None, 256)               65792     
_________________________________________________________________
dense_269 (Dense)            (None, 256)               65792     
_________________________________________________________________
dense_270 (Dense)            (None, 64)                16448     
_________________________________________________________________
dense_271 (Dense)            (None, 64)                4160      
_________________________________________________________________
dense_272 (Dense)            (None, 64)              

In [82]:
# predictions = np.zeros_like(predictions)
predictions.shape
predictions = (predictions>=0.5)
print(predictions)

[[ True]
 [False]
 [ True]
 ...
 [False]
 [False]
 [False]]


In [83]:
from sklearn.metrics import accuracy_score
accuracy_score(ValidationVectorLabels.astype(np.float),predictions)

0.6684415584415584